## Item 1
Deberá ejecutar tres algoritmos de clustering: K-Means, K-Means++ y
MeanShift, utilizando al menos cuatro configuraciones distintas para cada técnica
(deberá justificar la elección de parámetros). Para el entrenamiento, use únicamente
el 80% de los datos, omitiendo la etiqueta Y. A continuación, evalúe las doce
configuraciones obtenidas mediante una métrica de su elección (por ejemplo,
Silhouette Score) y seleccione las tres de mejor desempeño. Luego, aplique cada
una de estas configuraciones al 20% restante de los datos, asignando a cada muestra
el cluster correspondiente (obtenidos desde el entrenamiento). Finalmente, compare
la etiqueta Y real de cada muestra con la etiqueta dominante dentro del cluster al
que pertenece y analice si este procedimiento resulta razonable para asignar
etiquetas faltantes.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, accuracy_score, classification_report


In [ ]:
housing = fetch_california_housing()
x = housing.data
x = StandardScaler().fit_transform(x)
df_x = pd.DataFrame(x, columns=housing.feature_names)

y = housing.target
y = pd.qcut(y, q=4, labels=[0,1,2,3])

Q1 = df_x.quantile(0.25)
Q3 = df_x.quantile(0.75)
IQR = Q3 - Q1
mask = ((df_x >= (Q1 - 1.5 * IQR)) & (df_x <= (Q3 + 1.5 * IQR))).all(axis=1)

x = df_x[mask].values
y = y[mask]

train_data, test_data, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

df_housing = pd.DataFrame(train_data, columns=housing.feature_names)
df_price = pd.DataFrame(y_train, columns=['MedHouseVal'])

print(df_price.head())
print(df_housing.head())
print(df_price.shape)
print(df_housing.shape)

## Mean-shift

In [ ]:
from sklearn.cluster import estimate_bandwidth, get_bin_seeds, MeanShift

In [ ]:
qt = 0.06
nsamples = 10000
bandwidth = estimate_bandwidth(train_data, quantile=qt, n_samples=nsamples)
# quantile es el porcentaje de puntos que se consideran vecinos (Sensibilidad)

seeds = get_bin_seeds(train_data, bin_size=bandwidth, min_bin_freq=1)
#Ajusta la cantidad de puntos iniciales (semillas) para el algoritmo

# <-- Mean Shift Clustering -->
ms = MeanShift(bandwidth=bandwidth, seeds=seeds)
ms.fit(train_data)
cluster_centers = ms.cluster_centers_
labels = ms.labels_
n_clusters_ = len(np.unique(labels))
print("Number of estimated clusters: %d" % n_clusters_)

# <-- Evaluación del clustering -->
silhouette_vals = silhouette_score(train_data, labels)
print(f'Silhouette Score: {silhouette_vals}')

# <-- Mapeo de clusters a etiquetas reales -->
mapa_clusters = {}
print("Etiquetas reales:", np.unique(y_train))
for cluster_id in np.unique(labels):
    if cluster_id == -1:
        continue
    etiquetas_reales_en_cluster = y_train[labels == cluster_id] #Retorna array con las etiquetas reales de los puntos en el cluster
    print(f'Cluster {cluster_id}: etiquetas reales {etiquetas_reales_en_cluster}')
    etiqueta_dominante = mode(etiquetas_reales_en_cluster, keepdims=False)[0]
    print(f'Cluster {cluster_id}: etiqueta dominante {etiqueta_dominante}')
    mapa_clusters[cluster_id] = etiqueta_dominante
print("Mapa de clusters a etiquetas reales:", mapa_clusters)
   
cluster_test = ms.predict(test_data)
print("Clusters predichos para datos de prueba:", cluster_test)
y_pred = np.array([mapa_clusters.get(cluster_id, -1) for cluster_id in cluster_test])
print("Etiquetas predichas para datos de prueba:", y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print("Classification Report:")
print(classification_report(y_test, y_pred, zero_division=0))

In [ ]:
grafico_2d = PCA(n_components=2, random_state=42)
train_data_2d = grafico_2d.fit_transform(train_data)
centros_2d = grafico_2d.transform(cluster_centers)

plt.figure(figsize=(10,8))

plt.scatter(
    train_data_2d[:, 0],  # Eje X (Componente 1)
    train_data_2d[:, 1],  # Eje Z (Componente 3)
    c=labels,          # Color basado en el clúster
    cmap='viridis',
    s=10,
    alpha=0.5
)
plt.scatter(
    centros_2d[:, 0],
    centros_2d[:, 1],
    color='green',
    s=100,
    zorder=10,
    marker='o',
    label='Centros'
)

param_string = (
    f"Parámetros de MeanShift:\n"
    f"  Clústeres: {n_clusters_}\n"
    f"  Bandwidth: {bandwidth:.3f}\n"
    f"  Quantile: {qt}\n"
    f"  N_Samples: {nsamples}\n"
    f"  Numero de puntos iniciales: {len(train_data)}"
)

plt.text(
    -3.7, 2.5, param_string,
    ha='left', va='top',
    fontsize=7,
    bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.7)
)

plt.title('Resultados de MeanShift (visualizados con PCA 2D)')
plt.legend()
plt.show()

#

## K-Means

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
train_data_kmeans = pd.DataFrame(train_data)
kmeans = KMeans(n_clusters=8, random_state=42)
kmeans.fit(train_data_kmeans)
clusters_kmeans = kmeans.labels_
print(train_data_kmeans.head())
print(len(np.unique(clusters_kmeans)))

In [ ]:
grafico_2d_kmeans = PCA(n_components=2, random_state=42)
train_data_2d = grafico_2d_kmeans.fit_transform(train_data_kmeans)
centros_2d = grafico_2d_kmeans.transform(kmeans.cluster_centers_)
plt.figure(figsize=(10, 8))
plt.scatter(train_data_2d[:, 0], train_data_2d[:, 1],
            c=clusters_kmeans, cmap='viridis', s=10)
plt.scatter(
    centros_2d[:, 0],
    centros_2d[:, 1],
    color='red',
    s=100,
    zorder=10,
    marker='o',
    label='Centros'
)
param_string = (
    f"Parámetros de MeanShift:\n"
    f"  Clústeres: {n_clusters_}\n"
    f"  Bandwidth: {bandwidth:.3f}\n"
    f"  Quantile: {qt}\n"
    f"  N_Samples: {nsamples}\n"
    f"  Numero de puntos iniciales: {len(train_data)}"
)
plt.text(
    -3.7, 2.5, param_string,
    ha='left', va='top',
    fontsize=7,
    bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.7)
)
plt.legend()
plt.title('Clústeres KMeans en 2D (PCA)')
plt.colorbar(label='Clusters')
plt.show()